In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import re

In [15]:

pages = set()  
def getLinks_in(pageUrl):
                table=pd.DataFrame()
                html = urlopen('https://uk.wikipedia.org{}'.format(pageUrl))
                bs = BeautifulSoup(html, 'html.parser')
                try:
                    print(bs.h1.get_text())
                    print(bs.find(id='mw-pages').find_all('p')[0])
                except AttributeError:
                    print('This page is missing something! Continuing.')
                for link in bs.find(id='mw-pages').find_all('a', href=re.compile('^(/wiki/)')):
                    if 'href' in link.attrs:
                        if link.attrs['href'] not in pages:
                          # We have encountered a new page
                            newPage = link.attrs['href']
                            #print('-' * 20)
                            html = urlopen('https://uk.wikipedia.org{}'.format(newPage))
                            bs = BeautifulSoup(html, 'html.parser')
                            try:
                                city=bs.h1.text
                                print('city  ' + bs.h1.text)
                            except AttributeError:
                                city = None
                                #print('Error city')

                            # year
                            try:
                                year = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Заснован(е|о)')).find_parent('tr').find_all('td')[1].text.strip()
                                if  not year or year=='' or year=='невідомо' or year=='не відомо' or year=='—':
                                    #print ('i try')
                                    try:
                                        year = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Перша згадка')).find_parent('tr').find_all('td')[1].text.strip()
                                        #print('2 year  ' + year)
                                    except AttributeError:
                                        try:
                                            year = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Перша згадка')).find_parent('tr').find_all('td')[0].text.strip()
                                            #print('3 year  ' + year)
                                        except AttributeError:
                                            year = None  
                                    except IndexError:
                                        try:
                                            year = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Перша згадка')).find_parent('tr').find_all('td')[0].text.strip()
                                            #print('3 year  ' + year)
                                        except AttributeError:
                                            year = None        
                                else:   
                                    year
                                    #print('4 year  ' + year)


                            except IndexError:
                                #print('index error  ' + year)
                                try: 
                                    if not year or year=='' or year=='невідомо'  or year=='не відомо' or year=='—':
                                        year = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Заснован(е|о)')).find_parent('tr').find_all('td')[0].text.strip()
                                        #print('5 year  ' + year)
                                except AttributeError:
                                    year = None
                                except IndexError:
                                    if not year :
                                        year = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Перша згадка')).find_parent('tr').find_all('td')[0].text.strip()
                                        #print('6 year  ' + year)
                            except AttributeError:
                                try:
                                    year = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Перша згадка')).find_parent('tr').find_all('td')[1].text.strip()
                                    #print('7 year  ' + year)

                                except AttributeError:
                                    year = None        




                            # population
                            try:
                                population = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Населення')).find_parent('tr').find_all('td')[1].find(text=True, recursive=False).strip()
                                #print('population  ' + population)
                            except IndexError:
                                try:
                                    population = bs.find('table', {'class': 'infobox'}).find(text=re.compile('Населення')).find_parent('tr').find_all('td')[0].find(text=True, recursive=False).strip()
                                    #print('population  ' + population)
                                except AttributeError:
                                    population = None
                            except AttributeError:
                                population = None
                                #print('Error population')

                            # latitude
                            try:
                                latitude = bs.find('span', {'class': 'latitude'}).text
                                #print('latitude  ' + latitude)
                            except AttributeError:
                                try: 
                                    latitude = bs.find('a', {'class': 'mw-kartographer-maplink'}).text
                                    #latitude=latitude[:17]
                                    if latitude.find('″') == -1:
                                        latitude=latitude[:13]
                                    else: 
                                        latitude=latitude[:17]
                                except AttributeError:
                                    latitude = None


                            # longitude
                            try:
                                longitude = bs.find('span', {'class': 'longitude'}).text
                                #print('longitude 0 ' + longitude)
                            except AttributeError:
                                try:
                                    longitude = bs.find('a', {'class': 'mw-kartographer-maplink'}).text
                                    if longitude.find('″') == -1:
                                        longitude=longitude[13:]
                                    else: 
                                        longitude=longitude[17:]
                                except AttributeError:
                                    longitude = None

                            data= pd.DataFrame ({
                                "city": [city],
                                "year": [year],
                                "population": [population],
                                "latitude": [latitude],
                                "longitude": [longitude]

                            })
                            table=table.append(data,ignore_index=True)
                            pages.add(newPage)


                return table
def getLinks(pageUrl):
    global pages
    global table

    table=pd.DataFrame()
    html = urlopen('https://uk.wikipedia.org{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')                      
    for link in bs.find(id='mw-pages').find_all('a', href=re.compile('^(/w/index.php)'),text="наступна сторінка"):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                # We have encountered a new page
                newPage_main = link.attrs['href']
                print('------*' * 10)
   
               
        #getLinks_in(newPage_main)
        table=table.append(getLinks_in(newPage_main),ignore_index=True)
        #print('----step1-----')
        #print(table)
        #getLinks(newPage_main)
        table=table.append(getLinks(newPage_main),ignore_index=True)
        #print('----step2-----')
        #print(table)
            
    return table 


In [16]:
#міста
links=str('/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D1%96%D1%8F:%D0%9C%D1%96%D1%81%D1%82%D0%B0_%D0%A3%D0%BA%D1%80%D0%B0%D1%97%D0%BD%D0%B8')
#смт
#links=str('/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D1%96%D1%8F:%D0%A1%D0%B5%D0%BB%D0%B8%D1%89%D0%B0_%D0%A3%D0%BA%D1%80%D0%B0%D1%97%D0%BD%D0%B8')
#села
#links=str('/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D1%96%D1%8F:%D0%A1%D0%B5%D0%BB%D0%B0_%D0%A3%D0%BA%D1%80%D0%B0%D1%97%D0%BD%D0%B8')

In [17]:


table_pandas1=getLinks_in(links)
table_pandas2=getLinks(links)
#print('--------final-1--------')
#print(table_pandas1)
#print('--------final-2--------')
#print(table_pandas2)
#print('--------final-3--------')
table_pandas=table_pandas1.append(table_pandas2,ignore_index=True)
print(table_pandas)


Категорія:Міста України
<p>Показано 200 сторінок цієї категорії (із 486).
</p>
city  Міста України
city  Міста України (за алфавітом)
city  Місто районного значення
city  Міські агломерації України
city  Список міст України за покриттям додатків таксі
city  Історія міст України
city  Культурна столиця України
city  Міста України (за населенням)
city  Міста України (за часткою українців у населенні)
city  Нагороджені міста України
city  Шаблон:Найкращі міста для життя в Україні
city  Найкращі міста для життя в Україні за версією журналу «Фокус»
city  Перша столиця України
city  Столиці України
city  Авдіївка
city  Аджи-Мушкай
city  Алмазна
city  Алупка
city  Алушта
city  Алчевськ
city  Амвросіївка
city  Ананьїв
city  Андрушівка
city  Антрацит (місто)
city  Апостолове
city  Армянськ
city  Арциз
city  Балаклія
city  Балта
city  Бар (Україна)


KeyboardInterrupt: 

In [9]:
table_pandas

,city,year,population,latitude,longitude
860,Ясинувате (Вільнянський район),None,121,48°06′35″ пн. ш.,35°08′08″ сх. д.
861,Ясна Поляна (Василівський район),1795,88,47°22′48″ пн. ш.,35°07′15″ сх. д.
862,Ясна Поляна (Веселівський район),до 1932 р.,210,47°03′44″ пн. ш.,34°49′56″ сх. д.
863,Ясна Поляна (Оріхівський район),1868,623,47°48′38″ пн. ш.,35°45′05″ сх. д.
864,Ясне (Мелітопольський район),1921,511,47°01′41″ пн. ш.,35°38′55″ сх. д.


In [14]:
#міста
table_pandas.to_csv(r'C:/Users/Administrator/raw_data/raw_data_city.csv', index = False, header = True)
#смт
#table_pandas.to_csv(r'C:/Users/Administrator/raw_data_vilige_zaporiz2.csv', index = False, header = True)
#села
#table_pandas.to_csv(r'C:/Users/Administrator/raw_data_vilige.csv', index = False, header = True)
